In [1]:
from pandas_datareader import data as pdr 
import datetime as dt
from pandas_datareader import *
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import pandas_datareader.data as web
!pip install yfinance
!pip install googlefinance
from datetime import datetime
import pdb

In [2]:
end = dt.datetime.now()
start = dt.datetime(2000,1,1)

In [3]:
!pip install dash
import dash
from dash import html
from dash import dcc
!pip install pandas_datareader
!pip install matplotlib --upgrade

In [4]:
dataframe = pd.read_csv('nasdaq_screener_1693943158529.csv')

In [5]:
app = dash.Dash()

options = []

dataframe.info

<bound method DataFrame.info of      Symbol                                               Name Last Sale  \
0         A             Agilent Technologies Inc. Common Stock   $119.18   
1        AA                    Alcoa Corporation Common Stock     $30.22   
2       AAC  Ares Acquisition Corporation Class A Ordinary ...  $10.6952   
3      AACG   ATA Creativity Global American Depositary Shares     $1.23   
4      AACI            Armada Acquisition Corp. I Common Stock    $10.74   
...     ...                                                ...       ...   
7492   ZVSA             ZyVersa Therapeutics Inc. Common Stock   $0.1314   
7493    ZWS  Zurn Elkay Water Solutions Corporation Common ...   $29.085   
7494   ZYME                        Zymeworks Inc. Common Stock    $7.355   
7495   ZYNE          Zynerba Pharmaceuticals Inc. Common Stock     $1.33   
7496   ZYXI                            Zynex Inc. Common Stock     $7.10   

      Net Change % Change    Market Cap        Country 

In [12]:
for element in dataframe.index:

  options.append({"label": dataframe["Name"][element],
                  "value": dataframe["Symbol"][element]
                  })

app.layout = html.Div([
                       html.H1("Stock Ticker Web App"),
                       html.Div([
                                 html.H2("Select a stock:"),
                                 dcc.Dropdown(
                                     id = "dropdown",
                                     options = options,
                                     value = ["AAPL"],
                                     multi = True
                                 )
                       ]),
                       html.Div([
                                 html.H2("Select Date"),
                                 dcc.DatePickerRange(
                                     id = "datepicker",
                                     min_date_allowed = datetime(2000, 1, 1),
                                     max_date_allowed = datetime.today(),
                                     start_date = datetime(2020, 1, 1),
                                     end_date = datetime.today()
                                 )
                       ]),
                       html.Div([
                                 html.Button(
                                     id = "submit-button",
                                     n_clicks = 0,
                                     children = "Submit"
                                 )
                       ]),
                       dcc.Graph(
                           id = "stock-graph"
                       )
])

from dash.dependencies import Output, Input, State

import pandas

from pandas_datareader import DataReader

@app.callback(Output("stock-graph", "figure"),
              [Input("submit-button", "n_clicks")],
              [State("dropdown", "value"),
               State("datepicker", "start_date"),
               State("datepicker", "end_date")]
              )
def update_graph(number_of_clicks, stocks, start_date, end_date):

  start = datetime.strptime(start_date[:10], "%Y-%m-%d")

  end   = datetime.strptime(end_date[:10], "%Y-%m-%d")

  data = []

  for stock in stocks: 

    dataframe = DataReader(stock, "yahoo", start, end)

    dates = []

    for row in range(len(dataframe)):

      new_date = str(dataframe.index[row])

      new_date = new_date[0:10]

      dates.append(new_date)

    dataframe["Date"] = dates

    data.append({
        "x": dataframe["Date"],
        "y": dataframe["Adj Close"],
        "name": stock
    })

  figure = {
      "data": data,
      "layout": { 
          "title": "Stock Data"
      }
  }

  return figure

app.run_server(port=3030,)